In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow'

In [4]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow'

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [6]:
# Chargement des données
train_data = pd.read_csv('artifacts/data_training/train.csv') 
test_data = pd.read_csv('artifacts/data_training/test.csv') # Remplacez 'stb_data.csv' par votre fichier de données


In [8]:
import pandas as pd
import os
import joblib
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os

import joblib

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              AdaBoostClassifier, ExtraTreesClassifier)
from sklearn.linear_model import (LogisticRegression, LogisticRegressionCV, Perceptron)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score



# Remove 'tiers_key' and target 'Cluster' for X data
train_x = train_data.drop(columns=["tiers_key", "Cluster"])
test_x = test_data.drop(columns=["tiers_key", "Cluster"])

# Extract the target
train_y = train_data["Cluster"]
test_y = test_data["Cluster"]

# Encode all categorical features in the train and test datasets
def encode_features(train_df, test_df):
    # Combine train and test for consistent encoding
    combined = pd.concat([train_df, test_df], axis=0)
    
    # Encode categorical features
    for col in combined.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        combined[col] = le.fit_transform(combined[col].astype(str))
    
    # Split back into train and test sets
    train_encoded = combined.iloc[:len(train_df)]
    test_encoded = combined.iloc[len(train_df):]
    
    return train_encoded, test_encoded

train_x_encoded, test_x_encoded = encode_features(train_x, test_x)

# Encode the target labels
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(train_y)
test_y_encoded = label_encoder.transform(test_y)

# Standardize the features
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x_encoded)
test_x_scaled = scaler.transform(test_x_encoded)

# Initialize multiple models for evaluation
models = {
    "KNN": KNeighborsClassifier(n_neighbors=5, weights="distance"),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "SVC": SVC(kernel='linear', probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Gaussian Naive Bayes": GaussianNB(),
    "Perceptron": Perceptron(max_iter=1000, random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100, random_state=42),
    "Logistic Regression CV": LogisticRegressionCV(max_iter=200),
}

# Train and evaluate each model without PCA
accuracy_results_no_pca = {}

for model_name, model in models.items():
    model.fit(train_x_scaled, train_y_encoded)
    predictions_no_pca = model.predict(test_x_scaled)
    accuracy_no_pca = accuracy_score(test_y_encoded, predictions_no_pca)
    accuracy_results_no_pca[model_name] = accuracy_no_pca
    print(f"{model_name} Accuracy (No PCA): {accuracy_no_pca:.4f}")

# Perform cross-validation for the top models
top_models = {
    "Logistic Regression CV": LogisticRegressionCV(max_iter=200),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVC": SVC(kernel='linear', probability=True),
}

cv_results = {}

for model_name, model in top_models.items():
    cv_scores = cross_val_score(model, train_x_scaled, train_y_encoded, cv=5, scoring='accuracy')
    cv_results[model_name] = cv_scores.mean()
    print(f"{model_name} Cross-Validation Accuracy: {cv_scores.mean():.4f}")

# Display accuracy results without PCA
print("\nFinal Accuracy Results Without PCA:")
for model_name, accuracy in accuracy_results_no_pca.items():
    print(f"{model_name}: {accuracy:.4f}")

# Visualize the dataset with predicted labels
# Assuming you're using SVC for the prediction and you have 2 features for visualization
if train_x_scaled.shape[1] == 2:
    svc_model = SVC(kernel='linear')
    svc_model.fit(train_x_scaled, train_y_encoded)
    pred = svc_model.predict(test_x_scaled)

    # Plotting the data points
    plt.scatter(test_x_scaled[:, 0], test_x_scaled[:, 1], c=pred, cmap='coolwarm')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('SVC Predictions on Test Data')
    plt.colorbar()
    plt.show()
else:
    print("The dataset has more than 2 features, scatter plot cannot be generated.")


KNN Accuracy (No PCA): 0.9669
Random Forest Accuracy (No PCA): 0.9904
Logistic Regression Accuracy (No PCA): 0.8548
Gradient Boosting Accuracy (No PCA): 0.9932
AdaBoost Accuracy (No PCA): 0.9895
SVC Accuracy (No PCA): 0.8514
Decision Tree Accuracy (No PCA): 0.9886
Gaussian Naive Bayes Accuracy (No PCA): 0.7325
Perceptron Accuracy (No PCA): 0.8292
Extra Trees Accuracy (No PCA): 0.9904
Logistic Regression CV Accuracy (No PCA): 0.8554
Logistic Regression CV Cross-Validation Accuracy: 0.8449
Logistic Regression Cross-Validation Accuracy: 0.8439
SVC Cross-Validation Accuracy: 0.8430

Final Accuracy Results Without PCA:
KNN: 0.9669
Random Forest: 0.9904
Logistic Regression: 0.8548
Gradient Boosting: 0.9932
AdaBoost: 0.9895
SVC: 0.8514
Decision Tree: 0.9886
Gaussian Naive Bayes: 0.7325
Perceptron: 0.8292
Extra Trees: 0.9904
Logistic Regression CV: 0.8554
The dataset has more than 2 features, scatter plot cannot be generated.


In [ ]:
from sklearn.model_selection import cross_val_score

# Initialize a dictionary to hold cross-validation results
cv_results = {}

# Evaluate all models with cross-validation
for model_name, model in models.items():
    cv_scores = cross_val_score(model, train_x_scaled, train_y_encoded, cv=5, scoring='accuracy')
    cv_results[model_name] = cv_scores.mean()
    print(f"{model_name} Cross-Validation Accuracy: {cv_scores.mean():.4f}")

# Compare results
print("\nCross-Validation Accuracy Results:")
for model_name, accuracy in cv_results.items():
    print(f"{model_name}: {accuracy:.4f}")

    

Based on the cross-validation accuracy results you've provided, the Gradient Boosting model stands out as the best choice with a cross-validation accuracy of 0.9932. Here’s why:

1. Highest Cross-Validation Accuracy
Gradient Boosting has the highest cross-validation accuracy, indicating it performs better than all other models when generalizing to unseen data. This suggests it effectively captures the underlying patterns in the dataset.
2. Consistency
The high cross-validation score implies that Gradient Boosting is not likely overfitting. Models that perform well on training data but poorly on validation data might be overfitting, which can lead to poor performance in real-world applications. The high score across multiple folds of cross-validation indicates stability and robustness.
3. Comparison with Other Models
Other models, such as Extra Trees (0.9896) and Random Forest (0.9894), also exhibit high performance, but none reach the accuracy of Gradient Boosting. These models are also strong contenders, but their scores are slightly lower.
The KNN and AdaBoost models also perform well (0.9628 and 0.9642), but again, they do not match the performance of Gradient Boosting.
4. Importance in Practice
In practical applications, achieving a model that can predict accurately across diverse conditions is crucial. Since Gradient Boosting excels in both cross-validation and accuracy, it’s likely to provide reliable predictions when deployed.
Conclusion
Therefore, the final decision is to choose the Gradient Boosting model. Its superior performance in terms of cross-validation accuracy makes it the most suitable choice for your dataset and modeling objectives. If interpretability is not a primary concern, this model will provide the best predictive performance. However, if you require a balance between performance and interpretability, you might also consider Random Forest or Extra Trees, which offer strong accuracy with relatively better interpretability than Gradient Boosting.